In [1]:
# Parameters
# papermill: parameters
# tags: [parameters]

mode = None
input_path = None
output_path = None


In [2]:
# Parameters
mode = "fake"
input_path = ["C:\\store\\git\\km-stat-activity\\parquet_dataset\\date=2025-04-21\\profile_guid=011d3524-6fce-4184-bfa3-9707e8c6f18c\\part.6.parquet"]
output_path = "C:\\store\\git\\km-stat-activity\\processed\\fake\\profile_guid=011d3524-6fce-4184-bfa3-9707e8c6f18c\\2025-04-21-processed.csv"


In [3]:
import os
import pandas as pd
from pathlib import Path
import ast

# input_path'e göre veriyi oku
if isinstance(input_path, list):
    paths = [Path(p) for p in input_path]
    df = pd.concat([pd.read_parquet(p, engine="pyarrow") for p in paths], ignore_index=True)
else:
    df = pd.read_csv(str(input_path),
                     converters={"x": pd.eval, "y": pd.eval},
                     parse_dates=['start_date_time', 'end_date_time'])

# Bounding box hesaplama (örnek)
def get_bounding_box(x, y):
    if not x or not y:
        return (0, 0, 0, 0)
    return (min(x), min(y), max(x), max(y))

# bounding_box sütununu oluştur
df["bounding_box"] = df.apply(lambda row: get_bounding_box(row["x"], row["y"]), axis=1)

# bounding_box'tan min/max değerleri çıkar
df["min_x"] = df["bounding_box"].apply(lambda b: b[0])
df["min_y"] = df["bounding_box"].apply(lambda b: b[1])
df["max_x"] = df["bounding_box"].apply(lambda b: b[2])
df["max_y"] = df["bounding_box"].apply(lambda b: b[3])

# Sayısal tip dönüşümü için ondalık ayırıcı düzelt
cols = ["min_x", "min_y", "max_x", "max_y"]
for col in cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(",", ".", regex=False)
        .apply(lambda x: x if x.replace('.', '', 1).isdigit() else None)
        .astype(float)
    )


# bbox_area hesapla
df["bbox_area"] = (df["max_x"] - df["min_x"]) * (df["max_y"] - df["min_y"])



# Çıktıya tüm df'yi yaz
df_result = df

# CSV’ye yaz (birleştirerek)
if os.path.exists(output_path):
    df_existing = pd.read_csv(output_path)
    df_merged = pd.concat(
        [df_existing.reset_index(drop=True), df_result.reset_index(drop=True)],
        axis=1
    )
else:
    df_merged = df_result

df_merged.to_csv(output_path, index=False)
print(f"✔️ Saved CSV with full df including bounding box features: {output_path}")



✔️ Saved CSV with full df including bounding box features: C:\store\git\km-stat-activity\processed\fake\profile_guid=011d3524-6fce-4184-bfa3-9707e8c6f18c\2025-04-21-processed.csv
